# Clash Royale API to SQL Server ETL (Testing Notebook)

## I. Set up API and sql server connections

In [1]:
# import required libraries
import sys
import os
sys.path.append(os.path.abspath('..')) # Add project root (1 level up from notebook/)
from configs import config
import json
import pandas as pd
from datetime import datetime, timedelta
import time
from pytz import UTC
from sqlalchemy import create_engine, text
import requests

In [ ]:
# step up API access
api_key = config.API_KEY['Key']
headers = {'Authorization': f'Bearer {api_key}'}
base_url = 'https://api.clashroyale.com/v1'

In [3]:
with open('../configs/config.json') as f:
    config = json.load(f)

username = config['username']
password = config['password']
server = config['server']
database = config['database']

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&charset=utf8"
    "&autocommit=True"
)

engine = create_engine(connection_string, connect_args={"unicode_results": True})

## II. Create helper functions

### i. Season schedule functions

In [4]:
# create function to generate season id and date range for last n completed sns
def last_n_completed_seasons(n=3, ref_date=None):
    ref_date = ref_date or datetime.utcnow()

    # find the first monday of each month (day of season reset) going back
    months_back = n + 2
    first_mondays = []
    for i in range(months_back):
        m = (ref_date.month - i - 1) % 12 + 1  
        y = ref_date.year if (ref_date.month - i - 1) >= 0 else ref_date.year - 1 # handles cases of backtracking into previous year

        # checks first week of the month to find the first monday
        for d in range(1, 8):
            date = datetime(year=y, month=m, day=d)
            if datetime.weekday(date) == 0: # Monday is denoted as 0
                first_mondays.append(date)
                break
    first_mondays = sorted(first_mondays)

    # build seasons (start date, end date)
    seasons = []
    for i in range(len(first_mondays) - 1):
        start = first_mondays[i] + timedelta(hours=9) # specific start time for seasons
        end = first_mondays[i+1] + timedelta(hours=9) - timedelta(seconds=1)

        if end < ref_date:
            season_id = start.strftime('%Y-%m') # extract season_id from season start datetime
            seasons.append({'season_id': season_id,
                            'sn_start_date': start.replace(tzinfo=UTC),
                            'sn_end_date': end.replace(tzinfo=UTC)
            })
    return pd.DataFrame(seasons)
   

In [5]:
# create function to generate current and future season_id and date range
def current_plus_n_seasons(n=3, ref_date=None):
    ref_date = ref_date or datetime.utcnow()
    months_forward = n + 4
    
    first_mondays = []
    for i in range(-2, months_forward):
        m = (ref_date.month + i - 1) % 12 + 1
        y = ref_date.year + ((ref_date.month + i - 1) // 12)

        for d in range(1, 8):
            date = datetime(year=y, month=m, day=d)
            if datetime.weekday(date) == 0:
                first_mondays.append(date)
                break
    first_mondays = sorted(set(first_mondays))

    # build seasons as list of dicts with each dict representing a season row
    seasons = []
    
    for i in range(len(first_mondays) - 1):
        start = first_mondays[i] + timedelta(hours=9)
        end = first_mondays[i+1] + timedelta(hours=9) - timedelta(seconds=1)
        if start <= ref_date <= end: # add current season
            season_id = start.strftime('%Y-%m')
            seasons.append({'season_id': season_id,
                            'sn_start_date': start.replace(tzinfo=UTC),
                            'sn_end_date': end.replace(tzinfo=UTC)
            })
        elif start > ref_date and len(seasons) < n + 1:  # add up to n future
            seasons.append({'season_id': start.strftime('%Y-%m'),
                            'sn_start_date': start.replace(tzinfo=UTC),
                            'sn_end_date': end.replace(tzinfo=UTC)
            })   
    # return as df
    return pd.DataFrame(seasons)

In [6]:
# create function to find season_id based on battle_time
def battle_time_to_sid(battle_time, past_n=3, future_n=3):
    battle_time = pd.to_datetime(battle_time, utc =True)
    past_df = last_n_completed_seasons(n=past_n)
    future_df = current_plus_n_seasons(n=future_n)
    all_seasons_df = pd.concat([past_df, future_df], ignore_index=True).drop_duplicates(subset='season_id')
    
    for _, season in all_seasons_df.iterrows():
        if season['sn_start_date'] <= battle_time <= season['sn_end_date']:
            return season['season_id']
    return None

### ii. Existing data check functions

In [7]:
# create function to get existing values from any column of db table
def get_existing_data(engine, column, table): # column and table params will be strings
    with engine.connect() as conn:
        result = conn.execute(text(f'SELECT DISTINCT {column} FROM {table}'))
        return [getattr(row, column) for row in result]

# create function to get match_view_id:match_key mapping in matches db table
def get_match_key_mapping (engine):
    with engine.connect() as conn:
        match_key_map = pd.read_sql('SELECT match_view_id, match_key FROM matches;', conn)
        return match_key_map

### iii. Database insert (only) and delete functions

In [8]:
# create function to insert new rows of data into target db table; use for tables where existing rows are static data
def insert_new_rows(engine, source_df, target_table, method): # target_table and method params will be strings
    source_df.to_sql(name=target_table,
                     con=engine,
                     if_exists='append',
                     index=False, 
                     method=method
    )

# create function to delete rows from db (for deleted / banned players)
def purge_failed_players(engine, failed_players):
    with engine.begin() as conn:
        for player_id in failed_players:
            conn.execute(text("DELETE FROM match_cards WHERE player_id = :pid"), {"pid": player_id})
            conn.execute(text("DELETE FROM matches WHERE player_id = :pid"), {"pid": player_id})
            conn.execute(text("DELETE FROM season_rankings WHERE player_id = :pid"), {"pid": player_id})
            conn.execute(text("DELETE FROM players WHERE player_id = :pid"), {"pid": player_id})

### iv. Database upsert / merge functions

In [9]:
# create function to upsert (update existing + insert new) player data in players db table
def upsert_player_info(engine, df):
     player_info_tuples = df.values.tolist() # query expects column values passed as tuples
     
     query = '''
                MERGE INTO players AS target
                USING (VALUES (?,?,?,?,?,?,?,?,?,?,?)) AS 
                    source (player_id, player_name, exp_lvl, road_trophies, best_road_trophies, wins,
                            losses, life_time_battles, max_challenge_wins, clan_id, url_encoded_pid)
                ON target.player_id = source.player_id
                WHEN MATCHED THEN
                    UPDATE SET
                        player_name = source.player_name,
                        exp_lvl = source.exp_lvl,
                        road_trophies = source.road_trophies,
                        best_road_trophies = source.best_road_trophies,
                        wins = source.wins,
                        losses = source.losses,
                        life_time_battles = source.life_time_battles,
                        max_challenge_wins = source.max_challenge_wins,
                        clan_id = source.clan_id,
                        url_encoded_pid = source.url_encoded_pid
                WHEN NOT MATCHED THEN
                    INSERT (player_id, player_name, exp_lvl, road_trophies, best_road_trophies, wins, 
                            losses, life_time_battles, max_challenge_wins, clan_id, url_encoded_pid)
                        VALUES (source.player_id, source.player_name, source.exp_lvl, source.road_trophies,
                                source.best_road_trophies, source.wins, source.losses, source.life_time_battles,
                                source.max_challenge_wins, source.clan_id, source.url_encoded_pid);                                                            
     '''
     
     with engine.raw_connection().cursor() as cursor:
        cursor.executemany(query, player_info_tuples)
        cursor.connection.commit()

# create function to upsert clan data in clans db table
def upsert_clan_info(engine, df):
    clan_info_tuples = df.values.tolist()

    query = '''
                MERGE INTO clans AS target
                USING (VALUES (?,?,?,?,?,?,?,?,?,?)) AS
                    source (clan_id, clan_name, clan_type, badge_id, clan_score, clan_war_trophies,
                            clan_location, required_trophies, members, url_encoded_cid)
                ON target.clan_id = source.clan_id
                WHEN MATCHED THEN
                    UPDATE SET
                        clan_name = source.clan_name,
                        clan_type = source.clan_type,
                        badge_id = source.badge_id,
                        clan_score = source.clan_score,
                        clan_war_trophies = source.clan_war_trophies,
                        clan_location = source.clan_location,
                        required_trophies = source.required_trophies,
                        members = source.members,
                        url_encoded_cid = source.url_encoded_cid
                WHEN NOT MATCHED THEN
                    INSERT (clan_id, clan_name, clan_type, badge_id, clan_score, clan_war_trophies,
                            clan_location, required_trophies, members, url_encoded_cid)
                        VALUES (source.clan_id, source.clan_name, source.clan_type, source.badge_id, source.clan_score,
                                source.clan_war_trophies, source.clan_location, source.required_trophies,
                                source.members, source.url_encoded_cid);    
     '''
    with engine.raw_connection().cursor() as cursor:
        cursor.executemany(query, clan_info_tuples)
        cursor.connection.commit()

# create function to upsert card data in cards db table
def upsert_card_info(engine, df):
    card_info_tuples = df.where(pd.notnull(df), 0).values.tolist()

    query = '''
                MERGE INTO cards AS target
                USING (VALUES (?, ?, ?, ?, ?)) AS
                    source (card_id, card_name, rarity, elixir_cost, evo_status)
                ON target.card_id = source.card_id
                WHEN MATCHED THEN
                    UPDATE SET
                        card_name = source.card_name,
                        rarity = source.rarity,
                        elixir_cost = source.elixir_cost,
                        evo_status = source.evo_status
                WHEN NOT MATCHED THEN
                    INSERT (card_id, card_name, rarity, elixir_cost, evo_status)
                        VALUES (source.card_id, source.card_name, source.rarity, source.elixir_cost, source.evo_status);
    '''

    with engine.raw_connection().cursor() as cursor:
        cursor.executemany(query, card_info_tuples)
        cursor.connection.commit()

### iv. API data pull functions

In [10]:
# create function to pull player data into df
def get_player_info (player_ids):
    player_details = []
    failed_ids = [] # will catch any player_ids that fail to pull
    
    for player_id in player_ids:
        try:
            response = requests.get(f'{base_url}/players/{player_id}', headers=headers)

            if response.status_code != 200:
                print(f'Failed request for player {player_id.replace("%23", "#")}: {response.status_code}')
                failed_ids.append(player_id.replace('%23', '#'))
                continue
        
            json_data = response.json()
            if 'tag' not in json_data:
                print(f'Invalid response for player {player_id.replace("%23", "#")}: {json_data}')
                failed_ids.append(player_id.replace('%23', '#'))
                continue
        
            player_info = {'player_id': json_data['tag'],
                           'player_name': json_data['name'],
                           'exp_lvl': json_data['expLevel'],
                           'road_trophies': json_data['trophies'],
                           'best_road_trophies': json_data['bestTrophies'],
                           'wins': json_data['wins'],
                           'losses': json_data['losses'],
                           'life_time_battles': json_data['battleCount'],
                           'max_challenge_wins': json_data['challengeMaxWins'],
                           'clan_id': (json_data.get('clan') or {}).get('tag')
            }
            player_details.append(player_info)

        except Exception as e:
            print(f"Exception for player {player_id.replace('%23', '#')}: {e}")
            failed_ids.append(player_id.replace('%23', '#'))
        
        time.sleep(0.1) # stay under API limit
        
    players = pd.DataFrame(player_details)

    if not players.empty:
        players['url_encoded_pid'] = players['player_id'].str.replace('#', '%23')
        
    return players, failed_ids

In [11]:
# create function to pull season rankings from api into df
def get_season_rankings (season_ids):
    top_players = []
    with open('../dropped_data/dropped_players.json', 'r') as f:
        dropped_players = json.load(f) # load in dynamic list of known deleted or banned players to filter out

    for season in season_ids:
        response = requests.get(f'{base_url}/locations/global/pathoflegend/{season}/rankings/players?limit=100', headers=headers)
        if response.status_code != 200:
            print(f'Failed to fetch data for {season}.')
            continue # will continue run even if it fails to fetch data for a particular sn

        for player in response.json().get('items', []):
            if player['tag'] not in dropped_players: # apply filter                                                                                                    
                rank_info = {'player_id': player['tag'], # pull fields of interest
                            'season_id': season,
                            'rank': player['rank'],
                            'rating': player['eloRating']
                }
                top_players.append(rank_info)
                
    # create a DataFrame from the top players data
    season_rankings = pd.DataFrame(top_players)
    
    return season_rankings

In [12]:
# create function to pull clan data into df
def get_clan_info (clan_ids):
    clan_details = []
    failed_clans = []
    for clan_id in clan_ids:
        try:
            response = requests.get(f'{base_url}/clans/{clan_id}', headers=headers)
            if response.status_code != 200:
                    print(f'Failed request for clan {clan_id.replace("%23", "#")}: {response.status_code}')
                    failed_clans.append(clan_id.replace("%23", "#"))
                    continue
            
            json_data = response.json()
            if 'tag' not in json_data:
                print(f'Invalid response for clan {clan_id.replace("23", "#")}: {json_data}')
                failed_clans.append(clan_id.replace('23', '#'))
                continue

            clan_info = {'clan_id': json_data['tag'],
                        'clan_name': json_data['name'],
                        'clan_type': json_data['type'],
                        'badge_id': json_data['badgeId'],
                        'clan_score': json_data['clanScore'],
                        'clan_war_trophies': json_data['clanWarTrophies'],
                        'clan_location': json_data.get('location').get('name'),
                        'required_trophies': json_data['requiredTrophies'],
                        'members': json_data['members']
            }
            clan_details.append(clan_info)
            
        except Exception as e:
            print(f"Exception for clan {clan_id.replace('%23', '#')}: {e}")
            failed_clans.append(clan_id.replace('%23','#'))

        time.sleep(0.1)
        
    clans = pd.DataFrame(clan_details)
    clans['url_encoded_cid'] = clans['clan_id'].str.replace('#', '%23')
    clans['badge_id'] = clans['badge_id'].astype(str)  # ensure badge_id is string type
    
    return clans, failed_clans

In [13]:
# create function to pull card data into df
def get_card_info():
    card_info = []
    response = requests.get(f'{base_url}/cards', headers=headers)
    if response.status_code != 200:
        print(f'Failed to fetch card data: {response.status_code}')
        return pd.DataFrame()  # return empty DataFrame if request fails
    json_data = response.json().get('items', [])
    for card in json_data:
        card_details = {'card_id': card['id'],
                        'card_name': card['name'],
                        'rarity': card['rarity'],
                        'elixir_cost': card.get('elixirCost'), # elixir cost may not be present for all cards
                        'evo_status':  card.get('maxEvolutionLevel', 0) # evolutions have no level thus maxEvolutionLevel = 1 denotes evolution present
        }
        card_info.append(card_details)
        
    cards = pd.DataFrame(card_info)

    # table cleaning
    cards['evo_status'] = cards['evo_status'].astype(bool)
    cards['card_id'] = cards['card_id'].astype(str)
    cards['elixir_cost'] = cards['elixir_cost'].astype('Int64')
    cards['elixir_cost'] = cards['elixir_cost'].mask(cards['elixir_cost'].isnull(), None)
    
    return cards

In [14]:
# create function to pull matches data into df
def get_matches_info(player_ids): 
    matches_info = []
    failed_match_players = []
    raw_json_store = {}

    for player in player_ids:
        try:
            response = requests.get(f'{base_url}/players/{player}/battlelog', headers=headers)
            if response.status_code != 200:
                print(f'failed request for {player.replace("%23", "#")}: {response.status_code}')
                failed_match_players.append(player.replace('%23', '#'))
                continue

            json_data = response.json()
            raw_json_store[player] = json_data # want to also store json_data for later match_cards pull

            for match in json_data:
                team = match.get('team')[0]
                opp = match.get('opponent')[0]

                matches_details = {
                    'battle_time': match.get('battleTime'),
                    'game_mode': match.get('type'),
                    'league': match.get('leagueNumber'),
                    'player_id': team.get('tag'),
                    'opponent_id': opp.get('tag'),
                    'current_global_rank': team.get('globalRank'),
                    'starting_rating': team.get('startingTrophies'),
                    'rating_change': team.get('trophyChange'),
                    'crowns': team.get('crowns'),
                    'opp_crowns': opp.get('crowns'),
                    'king_tower_hp': team.get('kingTowerHitPoints'),
                    'princess_towers_hp': team.get('princessTowersHitPoints'),
                    'elixir_leaked': team.get('elixirLeaked')
                }

                matches_info.append(matches_details)

            time.sleep(0.1)  # stay under API rate limit

        except Exception as e:
            print(f'exception for {player.replace("%23", "#")}: {e}')
            failed_match_players.append(player.replace('%23', '#'))

    matches = pd.DataFrame(matches_info) # important note: each row is not necessarily a distinct match because two top players can play in the same match
                                         # each row is instead a distinct match-player combo; thus it is possible for the same match to be represented twice, once for each perspective
    if matches.empty:
        return matches, failed_match_players, raw_json_store

    # table cleaning
    matches['is_win'] = matches['crowns'] > matches['opp_crowns']
    matches['battle_time'] = pd.to_datetime(matches['battle_time'], format='%Y%m%dT%H%M%S.%fZ',
                                            utc=True, errors='coerce')
    matches['season_id'] = matches['battle_time'].apply(battle_time_to_sid)
    matches['match_key'] = (matches['battle_time'].astype(str) + '_' + matches['player_id'].astype(str)) # this col will be the checked col during ingestion
    matches = matches[matches['game_mode'] == 'pathOfLegend'].copy()
    matches['princess_tower1_hp'] = matches['princess_towers_hp'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 0)
    matches['princess_tower2_hp'] = matches['princess_towers_hp'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else 0)

    matches = matches[[
        'match_key','battle_time', 'is_win', 'league', 'player_id', 'opponent_id', 'season_id',
        'current_global_rank', 'starting_rating', 'rating_change', 'crowns','opp_crowns',
        'king_tower_hp', 'princess_tower1_hp', 'princess_tower2_hp', 'elixir_leaked',
    ]]

    return matches, failed_match_players, raw_json_store 


In [15]:
# create function to pull match cards (deck) data from stored json of battlelogs and into df
def get_match_card_info(raw_json):
    match_card_info = []
    for player in raw_json:
        for match in raw_json[player]:
            for card in match['team'][0]['cards']:
                match_card_details = {'game_mode': match['type'],
                                      'battle_time': match['battleTime'],
                                      'player_id': match['team'][0]['tag'],
                                      'card_id': card['id']
                }
                match_card_info.append(match_card_details)
    match_cards = pd.DataFrame(match_card_info)

    if match_cards.empty:
        return match_cards
    
    # table cleaning
    match_cards['battle_time'] = pd.to_datetime(match_cards['battle_time'], format='%Y%m%dT%H%M%S.%fZ', utc=True)
    match_cards['match_key'] = (match_cards['battle_time'].astype(str) + '_' + match_cards['player_id'].astype(str))
    match_cards['card_id'] = match_cards['card_id'].astype(str)
    match_cards = match_cards[match_cards['game_mode'] == 'pathOfLegend'].copy()
    match_cards.drop(columns=['game_mode', 'battle_time'], inplace=True)

    return match_cards # like matches, two perspectives of a match are possible

## III. Load data into database tables

In [16]:
# script to populate seasons table db table - must populate before all other tables
past_df = last_n_completed_seasons(n=3)
current_df = current_plus_n_seasons(n=0)
past_and_current_df = pd.concat([past_df, current_df], ignore_index=True).drop_duplicates(subset='season_id') # get current season and last 3 completed seasons

past_and_current_ids = past_and_current_df['season_id'].tolist()
existing_seasons = set(get_existing_data(engine, 'season_id', 'seasons')) # checking db for already stored seasons; convert to set for fast membership checking
all_new_seasons = [s for s in past_and_current_ids if s not in existing_seasons] # initial run will insert last 3 complete sns + current,
                                                                                 # subsequent runs insert newest, current sn, if not yet added 

if all_new_seasons:
    season_add_df = past_and_current_df[past_and_current_df['season_id'].isin(all_new_seasons)]
    print('New seasons to fetch:', all_new_seasons)
    insert_new_rows(engine, season_add_df, 'seasons', 'multi') # populate seasons with new seasons data
    print(f'Inserted {len(season_add_df)} new seasons.')
else:
    print('No new seasons to add.')

No new seasons to add.


In [17]:
# script to populate clans db table - must populate before players to respect db foreign key constraint (fk)
all_existing_past_seasons_ids = get_existing_data(engine, 'season_id', 'seasons')[:-1] # splice to remove current season
season_ranking_df = get_season_rankings(all_existing_past_seasons_ids)
tracked_players = season_ranking_df['player_id'].unique().tolist() # this will pick up players who can't ping api due to acc deletion or ban
tracked_players = [p.replace('#', '%23') for p in tracked_players]

player_df, failed_players = get_player_info(tracked_players)

with open('../dropped_data/dropped_players.json', 'r') as f:
    existing_dropped = json.load(f)

updated_dropped = list(set(existing_dropped + failed_players))

with open('../dropped_data/dropped_players.json', 'w') as f:
    json.dump(updated_dropped, f) # write failed players (banned or deleted players) into a dropped list to filter by in future runs
print(f'Added {len(failed_players)} player(s) into dropped list.')

current_membership = player_df['clan_id'].dropna().unique().tolist() # track clans top players are currently in
existing_clans = get_existing_data(engine, 'clan_id', 'clans') # track historical clans (where top players have been in) too
all_tracked_clans = list(set(current_membership + existing_clans))
all_tracked_clans = [c.replace('#', '%23') for c in all_tracked_clans]

clan_df, failed_clans = get_clan_info(all_tracked_clans)

if not clan_df.empty:
    upsert_clan_info(engine, clan_df)
    print(f'Upsert executed on {len(clan_df)} clan rows.')
else:
    print("No clans to upsert.")

if failed_clans:
    print(f"{len(failed_clans)} clan(s) failed to fetch. Retry later:")
    print(failed_clans)

Added 0 player(s) into dropped list.
Upsert executed on 175 clan rows.


In [18]:
# script to populate players db table - must populate before season_rankings to respect fk
upsert_player_info(engine, player_df)
print(f'Upsert executed on {len(player_df)} player rows.')

if failed_players:
    print(f"{len(failed_players)} player(s) failed to fetch. Already flagged for dropping:")
    print(failed_players)

Upsert executed on 234 player rows.


In [19]:
# script to populate season_rankings db tables
past_df = last_n_completed_seasons(n=3)
existing_season_rankings = set(get_existing_data(engine, 'season_id', 'season_rankings'))

past_ids = past_df['season_id'].tolist()
completed_new_seasons = [s for s in past_ids if s not in existing_season_rankings]

if completed_new_seasons:
    season_ranking_df = get_season_rankings(completed_new_seasons)
    insert_new_rows(engine, season_ranking_df, 'season_rankings', 'multi')
    print(f'Inserted {len(season_ranking_df)} new season rankings.')
else:
    print('No new seasons rankings to add.')

if failed_players:
    purge_failed_players(engine, failed_players) # need to delete any historical or recently added 
                                                 # data associated with dropped from all tables of db
    print(f'Removed all database records for {len(failed_players)} players:')
    print(failed_players)

No new seasons rankings to add.


In [20]:
# script to populate cards db table
cards_df = get_card_info() 
upsert_card_info(engine, cards_df)
print(f'Upsert executed on {len(cards_df)} card rows.')

Upsert executed on 120 card rows.


In [21]:
# script to populate matches and match_cards db tables
existing_rank_players = get_existing_data(engine, 'player_id', 'season_rankings')
existing_rank_players = {p.replace('#', '%23') for p in existing_rank_players}
match_logs, failed_players, raw_json_store = get_matches_info(existing_rank_players)

if match_logs.empty:
    print('No match data returned.')
else:
    existing_matches = set(get_existing_data(engine, 'match_key', 'matches'))
    match_keys = match_logs['match_key'].tolist()
    new_matches = [m for m in match_keys if m not in existing_matches]

    if new_matches:
        new_matches_df = match_logs[match_logs['match_key'].isin(new_matches)]
        unique_matches = new_matches_df.drop_duplicates(subset=['player_id', 'opponent_id'])
        cnt_unique_battles = len(unique_matches)

        insert_new_rows(engine, new_matches_df, 'matches', None) # must insert before match_cards to respect fk
        print(f'Inserted {len(new_matches_df)} new match views, of which {cnt_unique_battles} are unique matches.')
        
        # mapping the match_view_id identity int generated values in matches db table to match_cards_df rows
        match_key_mapping = get_match_key_mapping(engine)
        match_cards_df = get_match_card_info(raw_json_store)
        new_match_cards_df = match_cards_df[match_cards_df['match_key'].isin(new_matches)]
        new_match_cards_df = pd.merge(new_match_cards_df, match_key_mapping, on='match_key', how='inner')
        new_match_cards_df = new_match_cards_df[['match_view_id', 'player_id', 'card_id']]

        insert_new_rows(engine, new_match_cards_df, 'match_cards', None)
        print(f'Inserted {len(new_match_cards_df)} new rows into match_cards, spanning {int(len(new_match_cards_df)/ 8)} match views.')
    else:
        print('No new match data.')

if failed_players:
    print(f'{len(failed_players)} player(s) failed during match fetch.')

Inserted 135 new match views, of which 131 are unique matches.
Inserted 1080 new rows into match_cards, spanning 135 match views.
